In [2]:
from transformers import HubertModel
import torch
import librosa
import IPython.display as ipd
import os
import json

# To prevent the path from becoming corrupted when this cell is executed more than once.
try:
    path
except:
    path = "../"
    os.chdir(path)

from voice_cloning.BigVGAN.models import BigVGAN
from voice_cloning.BigVGAN.env import AttrDict
from voice_cloning.BigVGAN.meldataset import mel_spectrogram

In [4]:
reference_path = "sandbox/wavs/1_reference.wav"
vocoder_config_path = "voice_cloning/BigVGAN/checkpts/bigvgan-config.json"
vocoder_ckpt_path = "voice_cloning/BigVGAN/checkpts/bigvgan.pt"

In [6]:
print('Initializing Vocoder...')
with open(vocoder_config_path) as f:
    h = AttrDict(json.load(f))
vocoder = BigVGAN(h)
vocoder.load_state_dict(torch.load(vocoder_ckpt_path, map_location=lambda loc, storage: loc)['generator'])
_ = vocoder.cuda().eval()
vocoder.remove_weight_norm()

Initializing Vocoder...


/home/sadjava/anaconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...


In [7]:
# Preprocess the reference audio in a format suitable for fine-tuning
wav, sr = librosa.load(reference_path)
wav = torch.FloatTensor(wav).unsqueeze(0)
mel = mel_spectrogram(wav, 1024, 80, 22050, 256, 1024, 0.0, 8000.0, center=False)

# Load the normalization parameters for mel-spectrogram normalization.
mel_min = mel.min(-1, keepdim=True)[0]
mel_max = mel.max(-1, keepdim=True)[0]

with torch.no_grad():
    reference_audio = vocoder.forward(mel.cuda()).cpu().squeeze().clamp(-1, 1).numpy()

/home/sadjava/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [8]:
print('Reference audio')
ipd.display(ipd.Audio(reference_audio, rate=sr))

Reference audio
